In [1]:
from pathlib import Path
import torch

In [2]:
dataset_name = "TinyStories-train.txt"

In [3]:
file_path = Path.cwd() / '..' / 'data' / 'raw' / dataset_name
with open(file_path, 'r') as f:
    text = f.read()

# Explore the dataset

In [4]:
print(len(text))

1922767089


In [5]:
print(text[:1000])

One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.
Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."
Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.
<|endoftext|>
Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.
One day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were

# Tokenize the dataset
We will use characters as tokens, just as a baseline.

Note: special tokens already exist, like `<|endoftext|>`. We'll just ignore them for now, and generate infinite text.

Note: there appears to be different languages, symbols, and emojis. I can recognize Chinese characters. We'll ignore them for now.

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_`abcdefghijklmnopqrstuvwxyz{|}~ ¡¢£§«­°´·»¿ÂÉßàáâåèéêíïñóöúāİœɪʏʙʜіғᴀᴄᴅᴇᴏᴛᴜᴡᴢ   ​‌‎‐‑‒–—―‘’‚“”„…  ‪′€™−─❤　。」一了些他但保個們兒兩分到剛又和在天奮她己巴度很恩應把整是時會獨玉田留當的童答米給自興艾莉裡這過難高ﬁﬂ️﻿，￼�𝑐🌴🌹🍌🍞🎓💖🙂🤩
243


In [7]:
class TokenizerBase:
    def encode(self, s):
        raise NotImplementedError
    
    def decode(self, t):
        raise NotImplementedError


class CharacterTokenizer(TokenizerBase):
    def __init__(self, chars):
        self.stoi = { ch:i for i, ch in enumerate(chars) }
        self.itos = { i:ch for i, ch in enumerate(chars) }
    
    def encode(self, s):
        return [self.stoi[c] for c in s]
    
    def decode(self, l):
        return ''.join([self.itos[i] for i in l])

In [8]:
tokenizer = CharacterTokenizer(chars)
encoded = tokenizer.encode("Hii there")
print(encoded)
decoded = tokenizer.decode(encoded)
print(decoded)

[42, 74, 74, 2, 85, 73, 70, 83, 70]
Hii there


In [9]:
train_data = torch.tensor(tokenizer.encode(text[:1000]), dtype=torch.long)  # NOTE: just using first 1000 chars for now
print(train_data.shape, train_data.dtype)
print(train_data[:1000])

torch.Size([1000]) torch.int64
tensor([49, 79, 70,  2, 69, 66, 90, 14,  2, 66,  2, 77, 74, 85, 85, 77, 70,  2,
        72, 74, 83, 77,  2, 79, 66, 78, 70, 69,  2, 46, 74, 77, 90,  2, 71, 80,
        86, 79, 69,  2, 66,  2, 79, 70, 70, 69, 77, 70,  2, 74, 79,  2, 73, 70,
        83,  2, 83, 80, 80, 78, 16,  2, 53, 73, 70,  2, 76, 79, 70, 88,  2, 74,
        85,  2, 88, 66, 84,  2, 69, 74, 71, 71, 74, 68, 86, 77, 85,  2, 85, 80,
         2, 81, 77, 66, 90,  2, 88, 74, 85, 73,  2, 74, 85,  2, 67, 70, 68, 66,
        86, 84, 70,  2, 74, 85,  2, 88, 66, 84,  2, 84, 73, 66, 83, 81, 16,  2,
        46, 74, 77, 90,  2, 88, 66, 79, 85, 70, 69,  2, 85, 80,  2, 84, 73, 66,
        83, 70,  2, 85, 73, 70,  2, 79, 70, 70, 69, 77, 70,  2, 88, 74, 85, 73,
         2, 73, 70, 83,  2, 78, 80, 78, 14,  2, 84, 80,  2, 84, 73, 70,  2, 68,
        80, 86, 77, 69,  2, 84, 70, 88,  2, 66,  2, 67, 86, 85, 85, 80, 79,  2,
        80, 79,  2, 73, 70, 83,  2, 84, 73, 74, 83, 85, 16,  1, 46, 74, 77, 90,
         

# Create a dataloader

In [10]:
context_length = 8

In [11]:
class DataLoader:
    def __init__(self, context_length, batch_size, data):
        self.context_length = context_length
        self.batch_size = batch_size
        self.data = data
    
    def get_batch(self):
        ix = torch.randint(len(self.data) - self.context_length, (self.batch_size,))
        x = torch.stack([self.data[i:i+self.context_length] for i in ix])
        y = torch.stack([self.data[i+1:i+self.context_length+1] for i in ix])
        return x, y

In [15]:
torch.manual_seed(1337)
dataloader = DataLoader(context_length=8, batch_size=4, data=train_data)
xb, yb = dataloader.get_batch()
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(dataloader.batch_size):
    for t in range(dataloader.context_length):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[84, 73, 66, 83, 81, 16,  2, 46],
        [87, 74, 79, 72,  2, 74, 79,  2],
        [90,  2, 74, 79,  2, 85, 73, 70],
        [66, 83, 70,  2, 85, 73, 70,  2]])
targets:
torch.Size([4, 8])
tensor([[73, 66, 83, 81, 16,  2, 46, 74],
        [74, 79, 72,  2, 74, 79,  2, 85],
        [ 2, 74, 79,  2, 85, 73, 70,  2],
        [83, 70,  2, 85, 73, 70,  2, 79]])
----
when input is [84] the target: 73
when input is [84, 73] the target: 66
when input is [84, 73, 66] the target: 83
when input is [84, 73, 66, 83] the target: 81
when input is [84, 73, 66, 83, 81] the target: 16
when input is [84, 73, 66, 83, 81, 16] the target: 2
when input is [84, 73, 66, 83, 81, 16, 2] the target: 46
when input is [84, 73, 66, 83, 81, 16, 2, 46] the target: 74
when input is [87] the target: 74
when input is [87, 74] the target: 79
when input is [87, 74, 79] the target: 72
when input is [87, 74, 79, 72] the target: 2
when input is [87, 74, 79, 72, 2] the target: 74
when input is